In [2]:
#adapted from write-inputs-cluster (Ivan)
import numpy as np

In [9]:
def decimal_digits(va,vb,vc):
    maxn = 0

    for i in range(len(va)):
        if int(str(va[i])[::-1].find('.')) >= maxn:
            maxn = int(str(va[i])[::-1].find('.'))
        if int(str(vb[i])[::-1].find('.')) >= maxn:
            maxn = int(str(vb[i])[::-1].find('.'))
        if int(str(vc[i])[::-1].find('.')) >= maxn:
            
            maxn = int(str(vc[i])[::-1].find('.'))

    return maxn


In [13]:
def construct_lattice(va, vb, vc, size_x, size_y, size_z, maxn):

    positions = []

    for k in range(size_z):
        for i in range(size_x):
            for j in range(size_y):
                p = (np.multiply(va,i)+np.multiply(vb,j)+np.multiply(vc,k))
                positions.append(p.tolist())

    return np.round(positions, decimals=maxn).tolist()
    

In [59]:
def get_center_position(size_x, size_y, size_z, fx, fy, fz):

    mean_x = round(size_x*fx)
    mean_y = round(size_y*fy)
    mean_z = round(size_z*fz)

    center = size_x*size_y*mean_z + size_y*mean_x + mean_y + 1

    return center

In [60]:
def write_posfile_clus(data_imp, center, positions, NAME, maxn):

    print('Writing posfile_clus \n', end='')

    
    index_interactions = list(np.where(data_imp[:,0]==1)[0])       #index in list of cluster atoms
    data_imp_positions = data_imp[index_interactions,2:5]

    posclus = open('posfile_clus_'+NAME, 'w')

    all_sites_imp = [] #index in lattice of cluster atoms with center, ordered as firstly defined

    all_sites_imp.append(center)
    print('%3s%3s' %(1, 1)  + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*positions[center-1]), file = posclus)

    for i in range(len(data_imp_positions)):
        pos_now = np.add(positions[center-1],data_imp_positions[i]) #from center, finding position in lattice of cluster atoms
        pos_now = np.round(pos_now, decimals=maxn) 
        print('%3s%3s' %(i+2, i+2)  + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = posclus)
        
        ind = int(positions.index(list(pos_now)))+1 #finding index in lattice from position
        all_sites_imp.append(ind)

    posclus.close()

    print('[ok!]')
    return all_sites_imp

    

In [61]:
def write_xyz(all_sites_imp, positions, NAME, maxn):

    ### write a *.xyz file with 'Fe' format as the impurities and 'Nb' as the pristine
#   index order of atoms cluster respect lattice index order
    print('writing file .xyz \n', end='')
    
    system_file = open('system_'+NAME+'.xyz', 'w')

    print(str(len(positions)) + '\n', file = system_file)

    count = 1
    for i in range(len(positions)):
        pos_now = positions[i]
        pos_now = np.round(pos_now, decimals=maxn)
        if i+1 in all_sites_imp:
            print("Nb " + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = system_file)
            count += 1
        else:
            print("Fe" + '{0:12.6f} {1:12.6f} {2:12.6f}'.format(*pos_now), file = system_file)

    system_file.close()
    print('[ok!]')
    

In [71]:
def read_variables_from_file(filename):
    variables = {}
    
    with open(filename, 'r') as file:
        for line in file:
            # Strip leading/trailing whitespaces
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
            
            # Split by first space to separate key and value
            key, value = line.split(maxsplit=1)
            
            # Process value
            if value.startswith("[") and value.endswith("]"):
                # Convert to list of floats
                value = [float(x) for x in value.strip("[]").split(",")]
            elif "/" in value:
                # Handle fractions
                numerator, denominator = map(int, value.split("/"))
                value = numerator / denominator
            elif value.isdigit():
                # Convert integers
                value = int(value)
            else:
                # Convert to float
                value = float(value)
            
            # Assign to dictionary
            variables[key] = value
    
    return variables 

In [72]:
data_imp = np.loadtxt('jfile_clus')
variables = read_variables_from_file('lattice_inputs')

### Definitions ####
size_x = variables['size_x']
size_y = variables['size_y']
size_z = variables['size_z']
va = variables['va']
vb = variables['vb']
vc = variables['vc']
fx = variables['fx']
fy = variables['fy']
fz = variables['fz']

NAME=str(size_x)+'x'+str(size_y)+'x'+str(size_z)

maxn = decimal_digits(va, vb, vc)

positions = construct_lattice(va, vb, vc, size_x, size_y, size_z, maxn)

center = get_center_position(size_x, size_y, size_z, fx, fy, fz)

In [ ]:
all_sites_imp = write_posfile_clus(data_imp, center, positions, NAME, maxn)

write_xyz(all_sites_imp, positions, NAME, maxn)